In [9]:
import pickle

def read_dictionary(path):
    return pickle.load(open(path, 'rb'))


In [10]:
base_model_path = '../data/models'

In [13]:
import os
import pandas as pd
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from scipy.optimize import minimize
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor
pkl_files = [f for f in os.listdir(base_model_path) if f.endswith('.pkl')]
pkl_files

['rf_train_features.pkl',
 'meta_model.pkl',
 'rf_train_wide_features.pkl',
 'catboost_train_wide_features.pkl',
 'xgb_train_wide_features.pkl',
 'catboost_train_features.pkl',
 'lgbm_train_wide_features.pkl',
 'xgb_train_features.pkl',
 'lgbm_train_features.pkl']

In [14]:
arg_dict = {f[:-4]: read_dictionary(f'{base_model_path}/{f}') for f in pkl_files}
arg_dict.keys()

dict_keys(['rf_train_features', 'meta_model', 'rf_train_wide_features', 'catboost_train_wide_features', 'xgb_train_wide_features', 'catboost_train_features', 'lgbm_train_wide_features', 'xgb_train_features', 'lgbm_train_features'])

In [19]:
base_models = {
    'rf': RandomForestRegressor(**arg_dict['rf_train_features']),
    'catboost': CatBoostRegressor(**arg_dict['catboost_train_features']),
    'xgb': XGBRegressor(**arg_dict['xgb_train_features']),
    'lgbm': LGBMRegressor(**arg_dict['lgbm_train_features'])
}
base_wide_models = {
    'rf': RandomForestRegressor(**arg_dict['rf_train_wide_features']),
    'catboost': CatBoostRegressor(**arg_dict['catboost_train_wide_features']),
    'xgb': XGBRegressor(**arg_dict['xgb_train_wide_features']),
    'lgbm': LGBMRegressor(**arg_dict['lgbm_train_wide_features'])
}
meta_model = Ridge(alpha=arg_dict['meta_model']['alpha'], random_state=42)
meta_wide_model = Ridge(alpha=arg_dict['meta_model']['wide_alpha'], random_state=42)
wide_weight = arg_dict['meta_model']['wide_weight']

In [20]:
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)
stacking_model_wide = StackingRegressor(estimators=base_wide_models, final_estimator=meta_wide_model)